In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import scorecardpy as sc

In [ ]:
# Load your dataset (replace 'your_file.csv' with your actual file path)
df = pd.read_csv('C:/Users/Administrator/Documents/kifiya/Week_6/cleaned_data.csv')

## 1. Create Aggregate Features

In [ ]:
# Total Transaction Amount per customer
df['TotalTransactionAmount'] = df.groupby('CustomerId')['Amount'].transform('sum')

# Average Transaction Amount per customer
df['AverageTransactionAmount'] = df.groupby('CustomerId')['Amount'].transform('mean')

# Transaction Count per customer
df['TransactionCount'] = df.groupby('CustomerId')['TransactionId'].transform('count')

# Standard Deviation of Transaction Amounts per customer
df['TransactionAmountStd'] = df.groupby('CustomerId')['Amount'].transform('std').fillna(0)

## 2. Extract Time-Based Features

In [ ]:
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'], errors='coerce')

df['TransactionHour'] = df['TransactionStartTime'].dt.hour
df['TransactionDay'] = df['TransactionStartTime'].dt.day
df['TransactionMonth'] = df['TransactionStartTime'].dt.month
df['TransactionYear'] = df['TransactionStartTime'].dt.year

## 3. Encode Categorical Variables using WOE

In [ ]:
# 3. Encode Categorical Variables using scorecardpy's WOE
# ---------------------
# Assuming 'FraudResult' is the target variable and 'ProductCategory', 'ProviderId', 'ChannelId' are the features
features = ['ProductCategory', 'ProviderId', 'ChannelId']

# Calculate the WOE and IV for each feature
bins = sc.woebin(df, y='FraudResult', x=features)

# Apply the WOE transformation to the dataset
df_woe = sc.woebin_ply(df, bins)

## 4. Handle Missing Values

In [ ]:
# Fill missing numerical columns with median
for col in ['Amount', 'Value', 'TotalTransactionAmount', 'AverageTransactionAmount', 'TransactionAmountStd']:
    df_woe[col].fillna(df_woe[col].median(), inplace=True)

# Fill missing categorical columns with mode
for col in ['ProductCategory', 'CurrencyCode', 'ProviderId', 'ChannelId']:
    df_woe[col].fillna(df_woe[col].mode()[0], inplace=True)


## 5. Normalize/Standardize Numerical Features

In [ ]:
# Standardize (mean=0, std=1) the numerical features
scaler = StandardScaler()
df_woe[['Amount', 'Value', 'TotalTransactionAmount', 'AverageTransactionAmount', 'TransactionAmountStd']] = scaler.fit_transform(
    df_woe[['Amount', 'Value', 'TotalTransactionAmount', 'AverageTransactionAmount', 'TransactionAmountStd']]
)

In [ ]:
# Final check of the dataset
print(df_woe.head())

In [ ]:
# Save the processed dataset to CSV (optional)
df_woe.to_csv('C:/Users/Administrator/Documents/kifiya/Week_6/processed_data.csv', index=False)